In [2]:
import gradio as gr
from ultralytics import YOLO
from PIL import Image
import numpy as np

# Cargar modelo YOLO
char_model = YOLO("/home/gnz/GitHub/yolo11_container/YOLO_Characters/Character_YOLO_container_finetune_large/weights/best.pt")

def predict(image):
    # 1. Detección
    results = char_model.predict(image, conf=0.25)[0]  # Tomamos el primer batch

    # 2. Imagen con bounding boxes
    img_with_boxes = results.plot()
    img_with_boxes_pil = Image.fromarray(img_with_boxes)

    crops = []
    ocr_results = []

    # 3. Recorrer cada detección
    for box, cls, conf in zip(results.boxes.xyxy, results.boxes.cls, results.boxes.conf):
        x1, y1, x2, y2 = map(int, box)
        crop = image.crop((x1, y1, x2, y2))
        crops.append(crop)

        ocr_results.append({
            "class": int(cls),   # o str(results.names[int(cls)]) si querés nombres
            "confidence": float(conf)
        })

    return img_with_boxes_pil, crops, ocr_results


# 4. Interfaz Gradio
demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs=[
        gr.Image(type="pil", label="Detección IDs"),
        gr.Gallery(label="Crops con OCR", columns=2, height="auto"),     
        gr.JSON(label="Resultados OCR")
    ],
    title="Container Character Classification",
    description="Detecta caracteres de contenedores."
)

if __name__ == "__main__":
    demo.launch()


* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.



0: 448x640 2 0s, 1 1, 1 2, 1 4, 3 5s, 2 6s, 1 9, 1 G, 1 M, 1 U, 194.9ms
Speed: 5.2ms preprocess, 194.9ms inference, 2.9ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 3, 4 4s, 3 5s, 2 6s, 2 7s, 3 Cs, 1 S, 1 U, 141.9ms
Speed: 6.9ms preprocess, 141.9ms inference, 2.2ms postprocess per image at shape (1, 3, 448, 640)

0: 640x480 10 0s, 1 1, 20 2s, 5 3s, 9 4s, 4 6s, 4 7s, 4 8s, 6 9s, 2 As, 2 Bs, 6 Cs, 1 E, 1 F, 2 Gs, 1 I, 3 Ks, 1 M, 3 Rs, 2 Ss, 1 T, 2 Us, 1 X, 150.1ms
Speed: 10.2ms preprocess, 150.1ms inference, 6.7ms postprocess per image at shape (1, 3, 640, 480)

0: 448x640 2 0s, 1 1, 1 2, 1 4, 3 5s, 2 6s, 1 9, 1 G, 1 M, 1 U, 226.8ms
Speed: 6.0ms preprocess, 226.8ms inference, 2.5ms postprocess per image at shape (1, 3, 448, 640)
